In [47]:
!pip install tensorflow scikit-image opencv-python scikit-learn

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from skimage.feature import hog, local_binary_pattern
from skimage import exposure
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [2]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
# Flatten labels
y_train = y_train.ravel()
y_test = y_test.ravel()

In [4]:
# Class names for CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [5]:
# Display sample images
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(x_train[i])
    plt.title(class_names[y_train[i]])
    plt.axis('off')
plt.show()

In [6]:
# Preprocess images: Convert to grayscale and resize
def preprocess_images(images, size=(32, 32)):
    processed_images = []
    for img in images:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        resized_img = cv2.resize(gray_img, size)
        processed_images.append(resized_img)
    return np.array(processed_images)

x_train_processed = preprocess_images(x_train)
x_test_processed = preprocess_images(x_test)

In [7]:
# Normalize pixel values
x_train_processed = x_train_processed / 255.0
x_test_processed = x_test_processed / 255.0

In [8]:
# Display preprocessed images
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(x_train_processed[i], cmap='gray')
    plt.title(class_names[y_train[i]])
    plt.axis('off')
plt.show()

3. Feature Extraction

3.1 Histogram of Oriented Gradients (HOG)

In [9]:
# Extract HOG features
def extract_hog_features(images):
    hog_features = []
    for img in images:
        fd, hog_image = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, channel_axis=None)
        hog_features.append(fd)
    return np.array(hog_features)

x_train_hog = extract_hog_features(x_train_processed)
x_test_hog = extract_hog_features(x_test_processed)

In [10]:
# Display HOG features for a sample image
sample_hog, hog_image = hog(x_train_processed[0], pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, channel_axis=None)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(x_train_processed[0], cmap='gray')
plt.title('Original Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(hog_image, cmap='gray')
plt.title('HOG Features')
plt.axis('off')
plt.show()

 3.2 Local Binary Patterns (LBP)

In [11]:
# Extract LBP features
def extract_lbp_features(images, P=8, R=1):
    lbp_features = []
    for img in images:
        lbp = local_binary_pattern(img, P=P, R=R, method="uniform")
        hist, _ = np.histogram(lbp, bins=np.arange(0, P + 3), range=(0, P + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)  # Normalize
        lbp_features.append(hist)
    return np.array(lbp_features)

x_train_lbp = extract_lbp_features(x_train_processed)
x_test_lbp = extract_lbp_features(x_test_processed)

In [12]:
# Display LBP features for a sample image
sample_lbp = local_binary_pattern(x_train_processed[0], P=8, R=1, method="uniform")
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(x_train_processed[0], cmap='gray')
plt.title('Original Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(sample_lbp, cmap='gray')
plt.title('LBP Features')
plt.axis('off')
plt.show()

3.3 Edge Detection (Canny)

In [13]:
def extract_edge_features(images):
    edge_features = []
    for img in images:
        # Scale image back to [0, 255] and convert to 8-bit unsigned integer
        img_scaled = (img * 255).astype(np.uint8)
        edges = cv2.Canny(img_scaled, threshold1=100, threshold2=200)
        edge_features.append(edges.flatten())
    return np.array(edge_features)

x_train_edges = extract_edge_features(x_train_processed)
x_test_edges = extract_edge_features(x_test_processed)

In [14]:
# Display Edge features for a sample image
sample_edges = cv2.Canny((x_train_processed[0] * 255).astype(np.uint8), threshold1=100, threshold2=200)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(x_train_processed[0], cmap='gray')
plt.title('Original Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(sample_edges, cmap='gray')
plt.title('Edge Features')
plt.axis('off')
plt.show()

3.4 Deep Learning-Based Feature Extraction (ResNet)

In [15]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(32, 32, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)

In [16]:
# Preprocess images for ResNet
x_train_resnet = preprocess_input(x_train)
x_test_resnet = preprocess_input(x_test)

In [17]:
# Extract features
x_train_resnet_features = model.predict(x_train_resnet)
x_test_resnet_features = model.predict(x_test_resnet)

In [18]:
# Flatten features
x_train_resnet_features = x_train_resnet_features.reshape(x_train_resnet_features.shape[0], -1)
x_test_resnet_features = x_test_resnet_features.reshape(x_test_resnet_features.shape[0], -1)

4. Train and Evaluate Classifiers

4.1 Logistic Regression on HOG Features

In [19]:
# Train classifier
clf_hog = LogisticRegression(max_iter=1000)
clf_hog.fit(x_train_hog, y_train)

In [20]:
# Evaluate
y_pred_hog = clf_hog.predict(x_test_hog)
print("HOG + Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_hog))
print(classification_report(y_test, y_pred_hog, target_names=class_names))

In [21]:
# Confusion Matrix
cm_hog = confusion_matrix(y_test, y_pred_hog)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_hog, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix: HOG + Logistic Regression')
plt.show()

4.2 KNN on LBP Features

In [22]:
# Train KNN classifier on LBP features
clf_lbp = KNeighborsClassifier(n_neighbors=5)  # Define the KNN classifier
clf_lbp.fit(x_train_lbp, y_train)  # Train the classifier

In [23]:
# Evaluate the classifier
y_pred_lbp = clf_lbp.predict(x_test_lbp)  # Predict on the test data
print("LBP + KNN Accuracy:", accuracy_score(y_test, y_pred_lbp))  # Print accuracy
print(classification_report(y_test, y_pred_lbp, target_names=class_names))  # Print classification report

In [24]:
# Confusion Matrix
cm_lbp = confusion_matrix(y_test, y_pred_lbp)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_lbp, annot=True, fmt='d', cmap='Greens', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix: LBP + KNN')
plt.show()

4.3 Random Forest on Edge Features


In [25]:
# Train classifier
clf_edges = RandomForestClassifier(n_estimators=100)
clf_edges.fit(x_train_edges, y_train)

In [26]:
# Evaluate
y_pred_edges = clf_edges.predict(x_test_edges)
print("Edge Detection + Random Forest Accuracy:", accuracy_score(y_test, y_pred_edges))
print(classification_report(y_test, y_pred_edges, target_names=class_names))

In [27]:
# Confusion Matrix
cm_edges = confusion_matrix(y_test, y_pred_edges)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_edges, annot=True, fmt='d', cmap='Reds', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix: Edge Detection + Random Forest')
plt.show()

4.4 Logistic Regression on ResNet Features

In [28]:
# Train classifier
clf_resnet = LogisticRegression(max_iter=1000)
clf_resnet.fit(x_train_resnet_features, y_train)

In [29]:
# Evaluate
y_pred_resnet = clf_resnet.predict(x_test_resnet_features)
print("ResNet + Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_resnet))
print(classification_report(y_test, y_pred_resnet, target_names=class_names))

In [30]:
# Confusion Matrix
cm_resnet = confusion_matrix(y_test, y_pred_resnet)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_resnet, annot=True, fmt='d', cmap='Purples', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix: ResNet + Logistic Regression')
plt.show()

Hybrid Feature Fusion: HOG + ResNet

In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

In [32]:
# 1. Extract HOG features (as before)
x_train_hog = extract_hog_features(x_train_processed)  # (50000, n_hog_features)
x_test_hog = extract_hog_features(x_test_processed)    # (10000, n_hog_features)


In [33]:
# 2. Extract ResNet features (as before)
x_train_resnet_features = model.predict(x_train_resnet).reshape(x_train_resnet.shape[0], -1)
x_test_resnet_features = model.predict(x_test_resnet).reshape(x_test_resnet.shape[0], -1)

In [34]:
# 3. Combine features horizontally
x_train_hybrid = np.hstack([x_train_hog, x_train_resnet_features])  # (50000, n_hog + 2048)
x_test_hybrid = np.hstack([x_test_hog, x_test_resnet_features])     # (10000, n_hog + 2048)

In [35]:
# 4. Normalize and reduce dimensionality (optional but recommended)
pipeline = make_pipeline(
    StandardScaler(),
    PCA(n_components=128)  # Reduce to 128 dimensions for efficiency
)
x_train_hybrid = pipeline.fit_transform(x_train_hybrid)
x_test_hybrid = pipeline.transform(x_test_hybrid)

In [36]:
# 5. Train a classifier on hybrid features
clf_hybrid = LogisticRegression(max_iter=1000)  # Or use RandomForest/SVM
clf_hybrid.fit(x_train_hybrid, y_train)

In [37]:
# 6. Evaluate
y_pred_hybrid = clf_hybrid.predict(x_test_hybrid)
print("Hybrid (HOG+ResNet) Accuracy:", accuracy_score(y_test, y_pred_hybrid))
print(classification_report(y_test, y_pred_hybrid, target_names=class_names))

In [38]:
# Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix(y_test, y_pred_hybrid), 
            annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix: Hybrid (HOG+ResNet)')
plt.show()

Dynamic Feature Selection 

In [39]:
from tensorflow.keras.layers import Input, Dense, Multiply, Concatenate
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler

In [40]:
# 2. Separate normalization for each feature type
hog_scaler = StandardScaler()
resnet_scaler = StandardScaler()

x_train_hog_scaled = hog_scaler.fit_transform(x_train_hog)
x_train_resnet_scaled = resnet_scaler.fit_transform(x_train_resnet_features)

x_test_hog_scaled = hog_scaler.transform(x_test_hog)
x_test_resnet_scaled = resnet_scaler.transform(x_test_resnet_features)

In [41]:
# 3. Dynamic Feature Selection Model
def build_dynamic_fusion_model(hog_dim, resnet_dim, num_classes):
    hog_input = Input(shape=(hog_dim,), name='hog_input')
    resnet_input = Input(shape=(resnet_dim,), name='resnet_input')
    
    # Attention mechanism
    attention = Dense(hog_dim, activation='sigmoid', name='attention')(resnet_input)
    weighted_hog = Multiply(name='feature_weighting')([hog_input, attention])
    
    # Concatenate
    combined = Concatenate(name='feature_fusion')([weighted_hog, resnet_input])
    
    # Classification
    outputs = Dense(num_classes, activation='softmax', name='output')(combined)
    
    return Model(inputs=[hog_input, resnet_input], outputs=outputs)

In [42]:
# 4. Build model
model = build_dynamic_fusion_model(
    hog_dim=x_train_hog_scaled.shape[1], 
    resnet_dim=x_train_resnet_scaled.shape[1], 
    num_classes=10
)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [43]:
# 5. Train with validation
history = model.fit(
    [x_train_hog_scaled, x_train_resnet_scaled],
    y_train,
    epochs=15,
    batch_size=256,
    validation_data=([x_test_hog_scaled, x_test_resnet_scaled], y_test),
    verbose=1
)

In [44]:
# 6. Evaluation
test_loss, test_acc = model.evaluate([x_test_hog_scaled, x_test_resnet_scaled], y_test)
print(f"\nTest Accuracy: {test_acc:.4f}")

In [45]:
# 7. Visualization
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [46]:
# 8. Attention visualization (for one sample)
sample_idx = 10  # Change to see different examples
attention_model = Model(inputs=model.inputs, outputs=model.get_layer('attention').output)
attention_weights = attention_model.predict([x_test_hog_scaled[sample_idx:sample_idx+1], 
                                          x_test_resnet_scaled[sample_idx:sample_idx+1]])

plt.figure(figsize=(15, 4))
plt.bar(range(x_train_hog_scaled.shape[1]), attention_weights.flatten())
plt.title(f'Attention Weights for {class_names[y_test[sample_idx]]} (True) vs {class_names[np.argmax(model.predict([x_test_hog_scaled[sample_idx:sample_idx+1], x_test_resnet_scaled[sample_idx:sample_idx+1]]))]} (Predicted)')
plt.xlabel('HOG Feature Index')
plt.ylabel('Attention Weight')
plt.show()